In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import mglearn
%matplotlib inline

import os
path = 'E:\Python Consumer Credit'
os.chdir(path)

df_ml=pd.read_csv('ml_data.csv')

df_ml=df_ml.rename(columns={"expl": "target"})

target_count = df_ml.target.value_counts()
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

X_df = df_ml.iloc[:,:-1].values
y_df = df_ml.iloc[:,-1].values

#Undersampling
from imblearn.under_sampling import RandomUnderSampler as rus
us = rus(random_state=42)
X, y = us.fit_resample(X_df, y_df)

#One-hot encoding 
from sklearn.compose import ColumnTransformer             
from sklearn.preprocessing import OneHotEncoder
ct0 = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')  # Region
X = np.array(ct0.fit_transform(X))

ct1 = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[18])],remainder='passthrough')  # Currency
X = np.array(ct1.fit_transform(X))

ct2 = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[22])],remainder='passthrough')  # How
X = np.array(ct2.fit_transform(X))

ct3 = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[24])],remainder='passthrough')  # goal
X = np.array(ct3.fit_transform(X))

ct4 = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[26])],remainder='passthrough')  # gender
X = np.array(ct4.fit_transform(X))

ct5 = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[28])],remainder='passthrough')  # residence
X = np.array(ct5.fit_transform(X))

#Label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

#Train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,-3:] = sc.fit_transform(X_train[:,-3:])
X_test[:,-3:] = sc.transform(X_test[:,-3:])

from sklearn.decomposition import PCA
pca = PCA(n_components=33)
pca.fit_transform(X_train)
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

plt.figure(figsize=[10,7])
plt.plot(np.arange(1,34),pca.explained_variance_ratio_.cumsum())
plt.scatter(14,pca.explained_variance_ratio_.cumsum()[13],color='r',marker='+')
plt.xlim([1,33])
plt.ylim([0,1.05])
plt.title("PCA of undersampled data")
plt.xlabel("n_components",fontsize=14)
plt.ylabel("Explained variance ratio (Cumulative)",fontsize=14)
plt.grid()

In [ ]:
pca2 = PCA(n_components=14)
X_train = pca2.fit_transform(X_train)
X_test = pca2.transform(X_test)

ytr = y_train.reshape(y_train.shape[0],1)
yts = y_test.reshape(y_test.shape[0],1)

dat_tr = np.concatenate((X_train,ytr),axis=1)
dat_ts = np.concatenate((X_test,yts),axis=1)

np.savetxt("df__under_tr.csv", dat_tr, delimiter=",")
np.savetxt("df__under_ts.csv", dat_ts, delimiter=",")